In [69]:
import torch
import numpy as np
import torch.nn as nn
import sys
import os
project_root = os.path.abspath("..")  # Adjust if needed
import pytorch_lightning as pl
# Add the project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

from src.models.pointNetVae import PointNetVAE
from src.utils.data_utils import *
from src.dataset_classes.pointDataset import *
from proteinshake.datasets import ProteinFamilyDataset
from proteinshake.tasks import LigandAffinityTask

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
def test_chamfer_dist(x_1, x_2):
    min_a_b_i = 10000000
    total_loss = 0
    array = []
    array_num = []
    x_1 = np.array(x_1).squeeze()
    x_2 = np.array(x_2).squeeze()
    for a in x_1:
        
        array_2 = []
        for b_i in x_2:
            dist = np.sqrt(np.sum(np.abs(a - b_i)**2))
            array_2.append(dist)
            if dist < min_a_b_i:
                min_a_b_i = dist
        array.append(array_2)
        total_loss += min_a_b_i
        min_a_b_i = 100000

    for a in x_2:
        
        array_2 = []
        for b_i in x_1:
            dist = np.sqrt(np.sum(np.abs(a - b_i)**2))
            array_2.append(dist)
            if dist < min_a_b_i:
                min_a_b_i = dist
        array_num.append(array_2)
        total_loss += min_a_b_i
        min_a_b_i = 100000
    return total_loss, array, array_num

In [4]:
input1 = torch.randn(128, 500, 3)
input2 = torch.randn(128, 500, 3)
output = torch.cdist(input1, input2, p = 2)

In [5]:
output_array = []
for i in range(128):
    k = test_chamfer_dist(input1[i], input2[i])
    output_array.append(k[0])

In [8]:
optimizer = torch.optim.AdamW
optimizer_param = {'lr':0.001}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PointNetVAE(2, optimizer, optimizer_param)

In [60]:
from proteinshake.transforms import CenterTransform
dataset = ProteinFamilyDataset(root='../data').to_point().torch()
dataset = PointDataset(dataset, 500, False)

100%|██████████| 31109/31109 [00:04<00:00, 7357.91it/s]


In [63]:
dataset[0]

tensor([[ 34.5390, -11.4270, -11.5510,   2.0000],
        [ 32.8900, -11.7180,  -8.1200,  10.0000],
        [ 31.0270, -15.0550,  -8.3210,  18.0000],
        ...,
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000]])

In [58]:
torch.mean(dataset[0][:,:3], dim = 0)

tensor([ 15.3748,  -5.1807, -15.8877])

In [61]:
point_dataloader = DataLoader(dataset, batch_size = 128)

In [62]:
model(next(iter(point_dataloader)))

RuntimeError: Given groups=1, weight of size [64, 3, 1], expected input[128, 4, 500] to have 3 channels, but got 4 channels instead

In [22]:
dataset[0]

tensor([[ 34.5390, -11.4270, -11.5510,   2.0000],
        [ 32.8900, -11.7180,  -8.1200,  10.0000],
        [ 31.0270, -15.0550,  -8.3210,  18.0000],
        ...,
        [ 19.6510,   1.1720, -38.5240,   6.0000],
        [ 22.6870,   3.1040, -37.2450,  10.0000],
        [ 24.8080,   2.2730, -40.3010,  15.0000]])

In [37]:
batch_norm = nn.BatchNorm1d(3)

In [39]:
test = torch.randn(128,500,3)

In [42]:
batch_norm(test.permute(0,2,1)).shape

torch.Size([128, 3, 500])